In [13]:
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

In [19]:
from sklearn import svm
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

In [20]:
from math import sqrt
 
import torch
import torch.nn as nn
 
class SelfAttention(nn.Module):
    def __init__(self, dim_q, dim_k, dim_v):
        super(SelfAttention, self).__init__()
        self.dim_q = dim_q
        self.dim_k = dim_k
        self.dim_v = dim_v
 
        self.linear_q = nn.Linear(dim_q, dim_k,bias=False)
        self.linear_k = nn.Linear(dim_q, dim_k,bias=False)
        self.linear_v = nn.Linear(dim_q, dim_v,bias=False)
        self._norm_fact = 1 / sqrt(dim_k)
 
    def forward(self, x):
        batch, dim_q = x.shape
        n = 1
        assert dim_q == self.dim_q
        q = self.linear_q(x)
        k = self.linear_k(x)
        v = self.linear_v(x)

        dist = torch.mm(q, k.transpose(0,1)) * self._norm_fact
        
        dist = torch.softmax(dist, dim=-1)

        att = torch.mm(dist, v)
        return att

In [ ]:
torch.set_default_tensor_type(torch.DoubleTensor)
Best_rmse=10000
Best_RE=0
Best_F1=0
Best_FLAG=0
Best_ACC=0
LOW_ACC=1.1
LOW_RE=1.1
LOW_F1=1.1
LOW_FLAG=0
AVG_ACC=0
DF=0;
DB=0;
DV=0;
DM=0;
op_now=0
ACCNOW=[0 for i in range(825)]
fl=12
for iii in range(50,250,1):
    selector = SelectKBest(f_regression, k=iii)
    data=scio.loadmat('./floyd'+str(100-fl)+'_'+str(fl)+'.mat')
    X ={ }
    data_C ={ }
    data_W ={ }
    XXX_2d = [[0 for i in range(13858)]for j in range(161)]
    for i in range(161):
        X[i]=data['Z'][i]
    t=0
    for i in range(1,162):
        data_C[i]=scio.loadmat('./floyd'+str(100-fl)+'_'+str(fl)+'_'+str(i)+'.mat')
    data_W=scio.loadmat('./floyd'+str(100-fl)+'_'+str(fl)+'.mat')
    data_CC=np.array([[0 for i in range(164)]for j in range(161)],dtype='float')
    data_LL=np.array([0 for i in range(161)],dtype='float')
    data_WW=np.array([0 for i in range(161)],dtype='float')
    for i in range(1,162):
        for j in range(164):
            data_CC[i-1][j]=np.array(data_C[i]['C'][j],dtype='float')
    for i in range(161):
        data_WW[i]=np.array(data_W['Smallworld'][0][i],dtype='float')
    for i in range(161):
        t=0
        for k in range(0,164):
            for l in range(k+1,164):
                XXX_2d[i][t]=np.array(X[i][k,l])
                t=t+1
        for k in range(0,164):
            for l in range(164,167):
                if (l==164):
                    XXX_2d[i][t]=np.array(X[i][k,l])*data_CC[i][k]*data_WW[i]
                else:
                    XXX_2d[i][t]=np.array(X[i][k,l])
                t=t+1
    XX=np.array(XXX_2d,dtype='float')
    u,v=0,0;
    X_train =np.array([[0 for i in range(13858)] for j in range(138)],dtype='float')
    X_test =np.array([[0 for i in range(13858)] for j in range(23)],dtype='float')
    Y_train=np.array([0 for i in range(138)],dtype='float')
    Y_test=np.array([0 for i in range(23)],dtype='float')
    X_trainA =np.array([[0 for i in range(iii+13)] for j in range(138)],dtype='float')
    X_testA =np.array([[0 for i in range(iii+13)] for j in range(23)],dtype='float')
    Y_trainA=np.array([0 for i in range(138)],dtype='float')
    Y_testA=np.array([0 for i in range(23)],dtype='float')
    for j in range(161):
        if (j%7==op_now):
            X_test[u]=XX[j]
            Y_test[u]=YY[j]
            u=u+1;
        else:
            X_train[v]=XX[j]
            Y_train[v]=YY[j]
            v=v+1;
    for kkk in range(50):
        X_newR = selector.fit_transform(X_train,Y_train)
        X_newE = selector.transform(X_test)
        model=SelfAttention(161,iii+7,iii+7)
        X_newRA = np.array([[0 for i in range(iii+7)] for j in range(138)],dtype='float')
        X_newEA = np.array([[0 for i in range(iii+7)] for j in range(23)],dtype='float')
        Feature=np.zeros((iii+7,161))
        u,v=0,0;
        for i in range(161):
            if (i%7==op_now):
                for j in range(iii):
                    X_newEA[u][j]=X_newE[u][j]
                t=iii
                for j in range(7):
                    X_newEA[u][t]=np.array(X_2d[i][j],dtype=float)
                    t=t+1
                u=u+1
            else:
                for j in range(iii):
                    X_newRA[v][j]=X_newR[v][j]
                t=iii
                for j in range(7):
                    X_newRA[v][t]=np.array(X_2d[i][j],dtype=float)
                    t=t+1
                v=v+1
        for i in range(iii+7):
            u,v=0,0;
            for j in range(161):
                if (j%7==op_now):
                    Feature[i][j]=np.array(X_newEA[u][i])
                    u=u+1
                else:
                    Feature[i][j]=np.array(X_newRA[v][i])
                    v=v+1
        Feature_tensor = torch.from_numpy(Feature)
        YQ1=model(Feature_tensor)
        tagAA=torch.sum(YQ1, dim=1)
        u,v=0,0;
        for i in range(161):
            if (i%7==op_now):
                for j in range(iii+7):
                    X_testA[u][j]=X_newEA[u][j]*tagAA[j].detach().numpy()
                u=u+1
            else:
                for j in range(iii+7):
                    X_trainA[v][j]=X_newRA[v][j]*tagAA[j].detach().numpy()
                v=v+1
        ACC=0;
        cnt=0;
        clf_linear = svm.SVC(kernel='linear',C=0.095)
        clf_linear.fit(X_trainA,Y_train)
        score_linear_test = clf_linear.score(X_testA,Y_test)
        score_linear_train = clf_linear.score(X_trainA,Y_train)
        predict_test = clf_linear.predict(X_testA)
        ACC=ACC+score_linear_test;
        cnt=cnt+1;
        ACC=ACC/cnt
        AVG_ACC=AVG_ACC+ACC
        if ACC>Best_ACC:
            Best_FLAG=iii
            Best_ACC=ACC
        if ACC<LOW_ACC:
            LOW_FLAG=iii
            LOW_ACC=ACC
        print(iii,kkk,ACC)
        if (ACCNOW[iii]<ACC):
            ACCNOW[iii]=ACC
            PATH = "./ARZE"+str(iii)+"P0.pth"
            torch.save(model.state_dict(), PATH)
    print(iii,ACCNOW[iii])
print("BEST:")
print(Best_FLAG,Best_ACC)
print(LOW_FLAG,LOW_ACC)
print("ACC")